In [119]:
import subprocess
import json
from tqdm import tqdm
from multiprocessing import Pool
import itertools

In [3]:
resources = "AWS::EC2::CustomerGateway | AWS::EC2::EIP | AWS::EC2::Host | AWS::EC2::Instance | AWS::EC2::InternetGateway | AWS::EC2::NetworkAcl | AWS::EC2::NetworkInterface | AWS::EC2::RouteTable | AWS::EC2::SecurityGroup | AWS::EC2::Subnet | AWS::CloudTrail::Trail | AWS::EC2::Volume | AWS::EC2::VPC | AWS::EC2::VPNConnection | AWS::EC2::VPNGateway | AWS::EC2::RegisteredHAInstance | AWS::EC2::NatGateway | AWS::EC2::EgressOnlyInternetGateway | AWS::EC2::VPCEndpoint | AWS::EC2::VPCEndpointService | AWS::EC2::FlowLog | AWS::EC2::VPCPeeringConnection | AWS::Elasticsearch::Domain | AWS::IAM::Group | AWS::IAM::Policy | AWS::IAM::Role | AWS::IAM::User | AWS::ElasticLoadBalancingV2::LoadBalancer | AWS::ACM::Certificate | AWS::RDS::DBInstance | AWS::RDS::DBSubnetGroup | AWS::RDS::DBSecurityGroup | AWS::RDS::DBSnapshot | AWS::RDS::DBCluster | AWS::RDS::DBClusterSnapshot | AWS::RDS::EventSubscription | AWS::S3::Bucket | AWS::S3::AccountPublicAccessBlock | AWS::Redshift::Cluster | AWS::Redshift::ClusterSnapshot | AWS::Redshift::ClusterParameterGroup | AWS::Redshift::ClusterSecurityGroup | AWS::Redshift::ClusterSubnetGroup | AWS::Redshift::EventSubscription | AWS::SSM::ManagedInstanceInventory | AWS::CloudWatch::Alarm | AWS::CloudFormation::Stack | AWS::ElasticLoadBalancing::LoadBalancer | AWS::AutoScaling::AutoScalingGroup | AWS::AutoScaling::LaunchConfiguration | AWS::AutoScaling::ScalingPolicy | AWS::AutoScaling::ScheduledAction | AWS::DynamoDB::Table | AWS::CodeBuild::Project | AWS::WAF::RateBasedRule | AWS::WAF::Rule | AWS::WAF::RuleGroup | AWS::WAF::WebACL | AWS::WAFRegional::RateBasedRule | AWS::WAFRegional::Rule | AWS::WAFRegional::RuleGroup | AWS::WAFRegional::WebACL | AWS::CloudFront::Distribution | AWS::CloudFront::StreamingDistribution | AWS::Lambda::Function | AWS::NetworkFirewall::Firewall | AWS::NetworkFirewall::FirewallPolicy | AWS::NetworkFirewall::RuleGroup | AWS::ElasticBeanstalk::Application | AWS::ElasticBeanstalk::ApplicationVersion | AWS::ElasticBeanstalk::Environment | AWS::WAFv2::WebACL | AWS::WAFv2::RuleGroup | AWS::WAFv2::IPSet | AWS::WAFv2::RegexPatternSet | AWS::WAFv2::ManagedRuleSet | AWS::XRay::EncryptionConfig | AWS::SSM::AssociationCompliance | AWS::SSM::PatchCompliance | AWS::Shield::Protection | AWS::ShieldRegional::Protection | AWS::Config::ConformancePackCompliance | AWS::Config::ResourceCompliance | AWS::ApiGateway::Stage | AWS::ApiGateway::RestApi | AWS::ApiGatewayV2::Stage | AWS::ApiGatewayV2::Api | AWS::CodePipeline::Pipeline | AWS::ServiceCatalog::CloudFormationProvisionedProduct | AWS::ServiceCatalog::CloudFormationProduct | AWS::ServiceCatalog::Portfolio | AWS::SQS::Queue | AWS::KMS::Key | AWS::QLDB::Ledger | AWS::SecretsManager::Secret | AWS::SNS::Topic | AWS::SSM::FileData | AWS::Backup::BackupPlan | AWS::Backup::BackupSelection | AWS::Backup::BackupVault | AWS::Backup::RecoveryPoint | AWS::ECR::Repository | AWS::ECS::Cluster | AWS::ECS::Service | AWS::ECS::TaskDefinition | AWS::EFS::AccessPoint | AWS::EFS::FileSystem | AWS::EKS::Cluster | AWS::OpenSearch::Domain | AWS::EC2::TransitGateway | AWS::Kinesis::Stream | AWS::Kinesis::StreamConsumer | AWS::CodeDeploy::Application | AWS::CodeDeploy::DeploymentConfig | AWS::CodeDeploy::DeploymentGroup | AWS::EC2::LaunchTemplate | AWS::ECR::PublicRepository | AWS::GuardDuty::Detector | AWS::EMR::SecurityConfiguration | AWS::SageMaker::CodeRepository | AWS::Route53Resolver::ResolverEndpoint | AWS::Route53Resolver::ResolverRule | AWS::Route53Resolver::ResolverRuleAssociation | AWS::DMS::ReplicationSubnetGroup | AWS::DMS::EventSubscription | AWS::MSK::Cluster | AWS::StepFunctions::Activity | AWS::WorkSpaces::Workspace | AWS::WorkSpaces::ConnectionAlias | AWS::SageMaker::Model | AWS::ElasticLoadBalancingV2::Listener | AWS::StepFunctions::StateMachine | AWS::Batch::JobQueue | AWS::Batch::ComputeEnvironment | AWS::AccessAnalyzer::Analyzer | AWS::Athena::WorkGroup | AWS::Athena::DataCatalog | AWS::Detective::Graph | AWS::GlobalAccelerator::Accelerator | AWS::GlobalAccelerator::EndpointGroup | AWS::GlobalAccelerator::Listener | AWS::EC2::TransitGatewayAttachment | AWS::EC2::TransitGatewayRouteTable | AWS::DMS::Certificate | AWS::AppConfig::Application | AWS::AppSync::GraphQLApi | AWS::DataSync::LocationSMB | AWS::DataSync::LocationFSxLustre | AWS::DataSync::LocationS3 | AWS::DataSync::LocationEFS | AWS::DataSync::Task | AWS::DataSync::LocationNFS | AWS::EC2::NetworkInsightsAccessScopeAnalysis | AWS::EKS::FargateProfile | AWS::Glue::Job | AWS::GuardDuty::ThreatIntelSet | AWS::GuardDuty::IPSet | AWS::SageMaker::Workteam | AWS::SageMaker::NotebookInstanceLifecycleConfig | AWS::ServiceDiscovery::Service | AWS::ServiceDiscovery::PublicDnsNamespace | AWS::SES::ContactList | AWS::SES::ConfigurationSet | AWS::Route53::HostedZone"
resource_lists = resources.split(" | ")

In [26]:
proc = subprocess.Popen(
    ['aws', 'configservice', 'list-discovered-resources', '--resource-type', 'AWS::EC2::VPC'],
    stdout = subprocess.PIPE
    )
out, err = proc.communicate()
result = out.decode('utf-8')
result = json.loads(result)

In [27]:
result

{'resourceIdentifiers': [{'resourceType': 'AWS::EC2::VPC',
   'resourceId': 'vpc-03e53262faff38c42'}]}

In [25]:
result['resourceIdentifiers'][0]['resourceId']

'i-0303414e8cd5865c0'

### 리소스 확인

In [168]:
exist_resources = []

def find_resource(resource) -> str:    
#     for item in tqdm(resource):
    proc = subprocess.Popen(
        ['aws', 'configservice', 'list-discovered-resources', '--resource-type', f'{resource}'],
    stdout = subprocess.PIPE
    )
    out, err = proc.communicate()
    
    if 'resourceType' in str(out):
        out = out.decode('utf-8')
        out = json.loads(out)
        resourceID = out['resourceIdentifiers'][0]['resourceId']
        return [resource, resourceID]
    else:
        return 0


In [169]:
def multiprocessing(resource_lists) -> list:
    pool = Pool(processes=6)
    resource = pool.map(find_resource, resource_lists)
    exist_resources.append(resource)
#     print(resourceID)
    pool.close()
    pool.join()

In [170]:
multiprocessing(resource_lists)

In [176]:
exist_resources = list(itertools.chain(*exist_resources))
exist_resources = [item for item in exist_resources if item != 0]
exist_resources

[['AWS::EC2::Instance', 'i-0303414e8cd5865c0'],
 ['AWS::EC2::InternetGateway', 'igw-082d2f709fb46e0c1'],
 ['AWS::EC2::NetworkAcl', 'acl-09d6463116ca6874d'],
 ['AWS::EC2::NetworkInterface', 'eni-020dc0690f3e6a20e'],
 ['AWS::EC2::RouteTable', 'rtb-0bf37f5bb938ce786'],
 ['AWS::EC2::SecurityGroup', 'sg-0056aa002fd81c496'],
 ['AWS::EC2::Subnet', 'subnet-00411fd2c8840890e'],
 ['AWS::EC2::Volume', 'vol-0a6914dd0377cb8de'],
 ['AWS::EC2::VPC', 'vpc-03e53262faff38c42'],
 ['AWS::S3::Bucket', 'config-bucket-561123858318'],
 ['AWS::CodeDeploy::DeploymentConfig', '00000000-0000-0000-0000-000000000001'],
 ['AWS::Route53Resolver::ResolverRule',
  'rslvr-autodefined-rr-internet-resolver'],
 ['AWS::Route53Resolver::ResolverRuleAssociation',
  'rslvr-autodefined-assoc-vpc-03e53262faff38c42-internet-resolver'],
 ['AWS::Athena::WorkGroup', 'primary']]

### 세부 정보

In [178]:
proc = subprocess.Popen(
    ['aws', 'configservice', 'get-resource-config-history', '--resource-type', exist_resources[0][0], '--resource-id', exist_resources[0][1]],
    stdout = subprocess.PIPE
    )
out, err = proc.communicate()

In [180]:
result = out.decode('utf-8')
result = json.loads(result)
result

{'configurationItems': [{'version': '1.3',
   'accountId': '561123858318',
   'configurationItemCaptureTime': 1678192345.934,
   'configurationItemStatus': 'ResourceDiscovered',
   'configurationStateId': '1678192345934',
   'configurationItemMD5Hash': '',
   'arn': 'arn:aws:ec2:us-east-2:561123858318:instance/i-0303414e8cd5865c0',
   'resourceType': 'AWS::EC2::Instance',
   'resourceId': 'i-0303414e8cd5865c0',
   'awsRegion': 'us-east-2',
   'availabilityZone': 'us-east-2a',
   'resourceCreationTime': 1678110999.0,
   'tags': {'Name': 'test'},
   'relatedEvents': [],
   'relationships': [{'resourceType': 'AWS::EC2::Subnet',
     'resourceId': 'subnet-08cffe1b0ae1f9b88',
     'relationshipName': 'Is contained in Subnet'},
    {'resourceType': 'AWS::EC2::Volume',
     'resourceId': 'vol-0a6914dd0377cb8de',
     'relationshipName': 'Is attached to Volume'},
    {'resourceType': 'AWS::EC2::VPC',
     'resourceId': 'vpc-03e53262faff38c42',
     'relationshipName': 'Is contained in Vpc'},
 

In [201]:
config = result['configurationItems'][0]['configuration']
config = json.loads(config)
config

{'amiLaunchIndex': 0,
 'imageId': 'ami-0f3c9c466bb525749',
 'instanceId': 'i-0303414e8cd5865c0',
 'instanceType': 't2.micro',
 'kernelId': None,
 'keyName': None,
 'launchTime': '2023-03-06T13:56:39.000Z',
 'monitoring': {'state': 'disabled'},
 'placement': {'availabilityZone': 'us-east-2a',
  'affinity': None,
  'groupName': '',
  'partitionNumber': None,
  'hostId': None,
  'tenancy': 'default',
  'spreadDomain': None,
  'hostResourceGroupArn': None,
  'groupId': None},
 'platform': None,
 'privateDnsName': 'ip-172-31-0-134.us-east-2.compute.internal',
 'privateIpAddress': '172.31.0.134',
 'productCodes': [],
 'publicDnsName': 'ec2-18-222-113-217.us-east-2.compute.amazonaws.com',
 'publicIpAddress': '18.222.113.217',
 'ramdiskId': None,
 'state': {'code': 16, 'name': 'running'},
 'stateTransitionReason': '',
 'subnetId': 'subnet-08cffe1b0ae1f9b88',
 'vpcId': 'vpc-03e53262faff38c42',
 'architecture': 'x86_64',
 'blockDeviceMappings': [{'deviceName': '/dev/xvda',
   'ebs': {'attachTime

### DataFrame

In [206]:
df_config = pd.DataFrame.from_dict(config, orient='index')
df_config.T

,amiLaunchIndex,imageId,instanceId,instanceType,kernelId,keyName,launchTime,monitoring,placement,platform,...,metadataOptions,enclaveOptions,bootMode,platformDetails,usageOperation,usageOperationUpdateTime,privateDnsNameOptions,ipv6Address,tpmSupport,maintenanceOptions
0,0,ami-0f3c9c466bb525749,i-0303414e8cd5865c0,t2.micro,None,None,2023-03-06T13:56:39.000Z,{'state': 'disabled'},"{'availabilityZone': 'us-east-2a', 'affinity':...",None,...,"{'state': 'applied', 'httpTokens': 'optional',...",{'enabled': False},None,None,None,None,None,None,None,None
